In [9]:
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline
import numpy as np
import numexpr as ne
from datetime import datetime
import ipdb
# from scipy.ndimage import imread
%load_ext line_profiler
import import_ipynb

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [13]:
class Hexgrid():    
    def __init__(self, Nx, Ny, reposeAngle = np.deg2rad(0), dx=1, terrain=None):
        ################ Constants ######################
        self.g = 9.81 # Gravitational acceleration
        self.f = 0.04 # Darcy-Weisbach coeff
        self.a = 0.43 # Empirical coefficient
        self.rho_a = 0.5 # ambient density
        self.rho_j = np.array([1]) # List of current densities
        self.Nj = 1 # Number of sediment types
        
        
        ############## Input variables ###################
        self.Nx = Nx
        self.Ny = Ny
        self.dx = dx
        self.reposeAngle = reposeAngle
        
        ################     Grid       ###################
        self.X = np.zeros((Ny, Nx, 2)) # X[:,:,0] = X coords, X[:,:,1] = Y coords
        for j in range(Ny):
            self.X[j,:,0] = j*dx/2 + np.arange(Nx) * dx    
            self.X[j,:,1] = -np.ones(Nx) * dx*np.sqrt(3)/2 * j

        
        ################# Cell substate storage ####################        
        self.Q_a   = np.zeros((self.Ny,self.Nx)) # Cell altitude (bathymetry)
        self.Q_th  = np.zeros((self.Ny,self.Nx)) # Turbidity current thickness
        self.Q_v   = np.zeros((self.Ny,self.Nx)) # Turbidity current velocity
        self.Q_cj  = np.zeros((self.Ny,self.Nx,self.Nj)) # jth current sediment volume concentration
        self.Q_cbj = np.zeros((self.Ny,self.Nx,self.Nj)) # jth bed sediment volume fraction
        self.Q_d   = np.ones((self.Ny,self.Nx)) * np.inf # Thickness of soft sediment
        self.Q_d[1:self.Ny-1,1:self.Nx-1] = 0
        self.Q_o   = np.zeros((self.Ny,self.Nx)) # Density current outflow
        
        ################### Set Initial conditions #####################
        self.setBathymetry(terrain)
        self.diff = np.zeros((self.Ny-2,self.Ny-2,6))
        self.seaBedDiff = np.zeros((self.Ny-2,self.Nx-2,6))
        self.calc_bathymetryDiff()
        
        
        self.totalheight = self.Q_d + self.Q_a
        
        ################################################################
        ##########################  Methods ############################
        ################################################################
        
    def time_step(self):
#         g_prime = self.calc_g_prime()
#         dt = self.calc_dt()
        # The order comes from the article
#         self.T_1(dt) # Water entrainment. TODO: See below for details
#         self.T_2() # TODO: Erosion and deposition
#         self.I_1() # TODO: Turbidity c. outflows
#         self.I_2() # TODO: Update thickness and concentration
#         self.I_3() # TODO: Update of turbidity flow velocity
        self.I_4() # Toppling rule
        
        
    def T_1(self,dt): # Water entrainment. IN: Q_a,Q_th,Q_cj,Q_v. OUT: Q_vj,Q_th
        # TODO! Fix correct value of Richardson no. Ri(no flow) = 0 ?
#         ipdb.set_trace()
        g_prime = self.calc_g_prime()
        Ri = g_prime*self.Q_th/(self.Q_v**2) # Richardson number
        Ri[Ri==0] = np.inf
        E_wStar = 0.075/np.sqrt(1+718*Ri**(2.4)) # Dimensionless incorporation rate
        E_w = self.Q_v*E_wStar # Rate of seawater incorporation
        nQ_th = self.Q_th + E_w*dt # Update cell current thickness
        
        self.Q_cj = self.Q_cj*self.Q_th/nQ_th
        self.Q_th = nQ_th
        
        
    def T_2(self): # TODO: Erosion and deposition. IN: Q_a,Q_th,Q_cj,Q_cbj,Q_v. OUT: Q_a,Q_d,Q_cj,Q_cbj
        pass
    def I_1(self): # TODO: Turbidity c. outflows. IN: Q_a,Q_th,Q_v,Q_cj. OUT: Q_o
        pass
    def I_2(self): # TODO: Update thickness and concentration. IN: Q_th,Q_cj,Q_o. OUT: Q_th,Q_cj
        pass
    def I_3(self): # Update of turbidity flow velocity. IN: Q_a,Q_th,Q_o,Q_cj. OUT: Q_v
#         TODO: Er hastighet bare et tall? Hva er retningen!?
        ipdb.set_trace()
        g_prime = self.calc_g_prime()
        sum_q_cj = np.sum(self.Q_cj,axis=2) # TCurrent sediment volume concentration
        q_o = self.Q_o[1:-1,1:-1]
        self.calc_Hdiff()
        U_k = np.zeros((self.Ny-2,self.Nx-2,6))
        diff = self.diff.copy()
        diff[np.isinf(diff)] = 0
        
        for i in range(6):
            U_k[:,:,i] = np.sqrt(8*g_prime[1:-1,1:-1]*sum_q_cj[1:-1,1:-1]/(self.f*(1+self.a))) * np.sqrt(q_o*diff[:,:,i])
        
        # TODO! Average the velocities. BUT HOW?
#         self.Q_v[1:-1,1:-1,:] = U_k.copy()
    
    def I_4(self): # Toppling rule
        interiorH = self.Q_d[1:self.Ny-1,1:self.Nx-1]

        angle = np.zeros((self.Ny-2,self.Ny-2,6))
        indices = np.zeros((self.Ny-2,self.Ny-2,6))
        NoOfTrans = np.zeros((self.Ny-2,self.Nx-2))
        frac  = np.zeros((self.Ny-2,self.Nx-2,6))
        deltaS = np.zeros((self.Ny-2,self.Nx-2,6))
        deltaSSum = np.zeros((self.Ny-2,self.Nx-2))

        self.calc_Hdiff()
        diff = self.diff
        
        # Find angles
        dx = self.dx
        angle = ne.evaluate('arctan2(diff,dx)')

        # (Checks if cell (i,j) has angle > repose angle and that it has mass > 0. For all directions.)      
        # Find cells (i,j) for which to transfer mass in the direction given
        for i in np.arange(6):
            indices[:,:,i] = np.logical_and(angle[:,:,i]>self.reposeAngle, (interiorH > 0) )  # Gives indices (i,j) where the current angle > repose angle and where height is > 0


        # Count up the number of cells (i,j) will be transfering mass to. If none, set (i,j) to infinity so that division works.
    #         NoOfTrans = np.sum(indices,axis=2)  # Gir tregere resultat?
        for i in np.arange(6):
            NoOfTrans += indices[:,:,i]
        NoOfTrans[NoOfTrans == 0] = np.inf


        # Calculate fractions of mass to be transfered
        for i in np.arange(6):
            frac[(indices[:,:,i]>0),i] = (0.5 * (diff[(indices[:,:,i]>0),i] - self.dx * np.tan(self.reposeAngle))/(interiorH[(indices[:,:,i]>0)]))
        frac[frac>0.5] = 0.5


        for i in np.arange(6): 
            deltaS[(indices[:,:,i]>0),i] = interiorH[(indices[:,:,i]>0)] * frac[(indices[:,:,i]>0),i]/NoOfTrans[(indices[:,:,i]>0)] # Mass to be transfered from index [i,j] to index [i-1,j]      

        # Lag en endringsmatrise deltaSSum som kan legges til self.Q_d
        # Trekk fra massen som skal sendes ut fra celler
        deltaSSum = -np.sum(deltaS,axis=2)

        # Legg til massen som skal tas imot
        deltaSSum += np.roll(np.roll(deltaS[:,:,0],-1,0),0,1)
        deltaSSum += np.roll(np.roll(deltaS[:,:,1],-1,0),1,1)
        deltaSSum += np.roll(np.roll(deltaS[:,:,2],0,0),1,1) 
        deltaSSum += np.roll(np.roll(deltaS[:,:,3],1,0),0,1) 
        deltaSSum += np.roll(np.roll(deltaS[:,:,4],1,0),-1,1)
        deltaSSum += np.roll(np.roll(deltaS[:,:,5],0,0),-1,1)

        self.Q_d[1:self.Ny-1,1:self.Nx-1] += deltaSSum
        self.totalheight = self.Q_a + self.Q_d # Update total height
        if (self.Q_d < -1e-7).sum() > 0:
            ipdb.set_trace()
            print('height',self.Q_d[1,6])
            raise RuntimeError('Negative sediment thickness!')
            
    def setBathymetry(self, terrain):
        if terrain is not None:
            x = np.linspace(0, 100, self.Nx)
            y = np.linspace(0, 100, self.Ny)
            X = np.array(np.meshgrid(x, y))
            temp = np.zeros((self.Ny,self.Nx))
            if terrain is 'river':
                temp = 2*X[1,:] + 5*np.abs(X[0,:] - 50 + 10*np.sin(X[1,:]/10))
#                 temp = 2*self.X[:,:,1] + 5*np.abs(self.X[:,:,0] + 10*np.sin(self.X[:,:,1]/10))
                self.Q_a =  temp[::-1,:]  # BRUK MED RIVER
            elif terrain is 'pit':
                temp = np.sqrt((X[0,:]-50)*(X[0,:]-50)+(X[1,:]-50)*(X[1,:]-50))
                self.Q_a = 10*temp
       
        
    def calc_bathymetryDiff(self):
        self.seaBedDiff[:,:,0] = self.Q_a[1:-1,1:-1] - self.Q_a[0:self.Ny-2, 1:self.Nx-1]
        self.seaBedDiff[:,:,1] = self.Q_a[1:-1,1:-1] - self.Q_a[0:self.Ny-2, 2:self.Nx  ]
        self.seaBedDiff[:,:,2] = self.Q_a[1:-1,1:-1] - self.Q_a[1:self.Ny-1, 2:self.Nx  ]
        self.seaBedDiff[:,:,3] = self.Q_a[1:-1,1:-1] - self.Q_a[2:self.Ny  , 1:self.Nx-1]
        self.seaBedDiff[:,:,4] = self.Q_a[1:-1,1:-1] - self.Q_a[2:self.Ny  , 0:self.Nx-2]
        self.seaBedDiff[:,:,5] = self.Q_a[1:-1,1:-1] - self.Q_a[1:self.Ny-1, 0:self.Nx-2]

    def calc_Hdiff(self):
        old_height = self.Q_d        
        interiorH = old_height[1:-1,1:-1]
        # Calculate height differences of all neighbors
        self.diff[:,:,0] =  interiorH - old_height[0:self.Ny-2, 1:self.Nx-1] + self.seaBedDiff[:,:,0] 
        self.diff[:,:,1] =  interiorH - old_height[0:self.Ny-2, 2:self.Nx  ] + self.seaBedDiff[:,:,1]
        self.diff[:,:,2] =  interiorH - old_height[1:self.Ny-1, 2:self.Nx  ] + self.seaBedDiff[:,:,2]
        self.diff[:,:,3] =  interiorH - old_height[2:self.Ny  , 1:self.Nx-1] + self.seaBedDiff[:,:,3]
        self.diff[:,:,4] =  interiorH - old_height[2:self.Ny  , 0:self.Nx-2] + self.seaBedDiff[:,:,4]
        self.diff[:,:,5] =  interiorH - old_height[1:self.Ny-1, 0:self.Nx-2] + self.seaBedDiff[:,:,5]
        

            
    def calc_rho_c(self): # out: current density rho_c matrix (all cells)
        sum = 0
        for j in range(self.Nj):
            sum += self.rho_j[j]*self.Q_cj[:,:,j]
        return self.rho_a*(1-np.sum(self.Q_cj,axis=2))+sum
    
    def calc_g_prime(self): # out: reduced gravity matrix (all cells)
#         ipdb.set_trace()
        sum = 0
        for j in range(self.Nj):
            sum += self.Q_cj[:,:,j]*(self.rho_j[j]-self.rho_a)/self.rho_a
        return self.g*sum

    def calc_potEnergy(self, g_prime): # out: potential energy matrix
#         ipdb.set_trace()
        A = 2*np.sqrt(3)*(self.dx/2)**2 # Area of hexagon = 2sqrt(3)*apothem
        rho_c = self.calc_rho_c()
        return rho_c*g_prime*A/2*(self.Q_th)**2
    
    def calc_BFroudeNo(self, g_prime): # out: Bulk Froude No matrix
#         g_prime = self.calc_g_prime()
        U = self.calc_potEnergy(g_prime)
        g_prime[g_prime==0] = np.inf 
        return 0.5*U**2/g_prime
    
    def calc_RunUpHeight(self, g_prime): # out: Run up height matrix
        h_k = self.calc_BFroudeNo(g_prime)
        return self.Q_th + h_k
    
    def calc_MaxRelaxationTime(self): #out: matrix
        g_prime = self.calc_g_prime()
        r_j = self.calc_RunUpHeight(g_prime)
        r_j[r_j == 0] = np.inf
        g_prime[g_prime==0] = np.inf
        return (self.dx/2)/np.sqrt(2*r_j*g_prime)
    
    def calc_dt(self):
        temp = self.calc_MaxRelaxationTime()
        return np.min(temp[np.nonzero(temp)])
    
    
        

        


In [14]:
# # Set initial position of mass
x = 5
y = 5

grid = Hexgrid(10,10,reposeAngle = np.deg2rad(0), terrain = None)
grid.Q_d[x,y] = 50
# grid.Q_th[y,x] = 1
# grid.Q_cj[y,x] = 1
# grid.Q_a[y,x] = 1
grid.time_step()
# print(grid.Q_th)
# grid.calc_dt()
# Nt =20
# grid.height[y, x] += 5000
# timeseries = np.zeros(Nt)
# for i in range(Nt):
#     grid.update_heightNew()
#     grid.calc_T_flow_velocity()


# fig = plt.figure(figsize = (9,9))
# ax = fig.add_subplot(111, aspect = 'equal')
# points = ax.scatter(grid.X[:,:,0].flatten(), grid.X[:,:,1].flatten(), marker = 'h', c = grid.seaBed.flatten())
# fig.colorbar(points,fraction=0.026)
# ax.set_title('Terrain(x,y)')

# ax.scatter(grid.X[y, x,0], grid.X[y, x,1]) # Targeting


# len(grid.X[:,:,0].flatten())

In [7]:
# fig = plt.figure(figsize = (9,9))
# ax = fig.add_subplot(111, aspect = 'equal')
# points = ax.scatter(grid.X[:,:,0].flatten(), grid.X[:,:,1].flatten(), marker = 'h', c = grid.height[:,:].flatten())
# fig.colorbar(points,fraction=0.026)
# ax.set_title('height(x,y)')

In [8]:
# fig = plt.figure(figsize = (9,9))
# ax = fig.add_subplot(111, aspect = 'equal')
# points = ax.scatter(grid.X[:,:,0].flatten(), grid.X[:,:,1].flatten(), marker = 'h', c = np.log(1+grid.Q_v[:,:,0].flatten()))
# fig.colorbar(points,fraction=0.026)
# ax.set_title('height(x,y)')